In [1]:

import time
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [177]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
# import seaborn as sns
# sns.set()
from sklearn import tree
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import seaborn as sns
from imblearn.over_sampling import SMOTE, ADASYN

In [485]:
filenames_merged = {
    "wisconsin_ssd_merged": "./ds/v3/selected_cols_merged/wisconsin-220g2-10Gbps_ssd_merged_V3.csv",
    # "wisconsin_ssd_unmerged": "./ds/v3/selected_cols/wisconsin-220g2-10Gbps_ssd_unmerged_V3.csv",

    "wisconsin_hdd_merged": "./ds/v3/selected_cols_merged/wisconsin-220g2-10Gbps_hdd_merged_V3.csv",
    # "wisconsin_hdd_unmerged": "./ds/v3/selected_cols/wisconsin-220g2-10Gbps_hdd_unmerged_V3.csv",

    "wisconsin_hdd_ssd_merged": "./ds/v3/selected_cols_merged/wisconsin-220g2-hdd-ssd_merged_V3.csv",
    # "wisconsin_hdd_ssd_unmerged": "./ds/v3/selected_cols/wisconsin-220g2-hdd-ssd_unmerged_V3.csv",

    "wisconsin_ssd_delay_10ms_merged":"./ds/v3/selected_cols_merged/wisconsin-220g2-ssd-delayed-10ms_merged_V3.csv",
    # "wisconsin_ssd_delayed_10ms_unmerged":"./ds/v3/selected_cols_merged/wisconsin-220g2-ssd-delayed-10ms_unmerged_V3.csv"

    "wisconsin_hdd_delay_10ms_merged":"./ds/v3/selected_cols_merged/wisconsin-220g2-hdd-delayed-10ms_merged_V3.csv",
    # # "wisconsin_hdd_delay_10ms_unmerged":"./ds/v3/selected_cols/wisconsin-220g2-hdd-delayed-10ms_unmerged_V3.csv",

    "utah_ssd_merged": "./ds/v3/selected_cols_merged/utah-6525-25g-25Gbps_ssd_merged.csv",
    # "utah_ssd_unmerged": "./ds/v3/selected_cols/utah-6525-25g-25Gbps_ssd_unmerged_V3.csv",

    "utah_ssd_delay_30ms_merged":"./ds/v3/selected_cols_merged/utah-6525-25-ssd-delayed-30ms_merged_V3.csv",
    # "utah_ssd_delay_unmerged": "./ds/v3/selected_cols/utah-6525-25-ssd-delayed-30ms_unmerged_V3.csv",


    "utah_ssd_delay_10ms_merged":"./ds/v3/selected_cols_merged/utah-6525-25-ssd-delayed-10ms_merged_V3.csv",
    # # "utah_ssd_delay_10ms_merged": "./ds/v3/selected_cols/utah-6525-25-ssd-delayed-10ms_unmerged_V3.csv",
}
data = {}

In [486]:
def read_file(transform=False):
    global data, filenames_merged

    for key in filenames_merged:
        df = pd.read_csv(filenames_merged[key])
        df = df.sort_values(by=['label_value']).reset_index().drop(['index','time_stamp',], axis=1)
        df.fillna(df.groupby(['label_value'], as_index=False).mean(), inplace=True)

        if transform:
            for index, row in features.iterrows():
                df[row['feature']] = df[row['feature']] / df[row['normalizer']]

        df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
        data[key] = df

read_file()

In [515]:
simul_data_names = {
    "wisconsin-220g2-ssd-simultanious-v2_merged":"./ds/v3/selected_cols/wisconsin-220g2-ssd-simultanious-v2.csv",
}
simul_data = {}

def read_file(transform=False):
    global data, filenames_merged

    for key in simul_data_names:
        df = pd.read_csv(simul_data_names[key])
        df = df.sort_values(by=['label_value']).reset_index().drop(['index','time_stamp',], axis=1)
        df.fillna(df.groupby(['label_value'], as_index=False).mean(), inplace=True)
        df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
        simul_data[key] = df

read_file()

In [5]:
def remove_labels_in_df(df, labels_values):
    for lbl in labels_values:
        df = df.drop(df[df.label_value == lbl].index)
    return df

In [6]:
GB = 1024 * 1024 * 1024

In [124]:
def check_receiver_ost_congestion(df):
    normal_ave_receiver_remote_ost_write_bytes = np.average(df[df.label_value == 0]['receiver_remote_ost_write_bytes'])
    normal_ave_receiver_nic_send = np.average(df[df.label_value == 0]['receiver_nic_send_bytes'])

    diff_from_normal_ost_write = normal_ave_receiver_remote_ost_write_bytes - df['receiver_remote_ost_write_bytes']  <= (0.3 * normal_ave_receiver_remote_ost_write_bytes)
    diff_from_client_remote_ost_write_bytes = df['receiver_write_bytes'] <= 0.9 * df['receiver_remote_ost_write_bytes']
    packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] < 0.05)
    diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] < 1.1 * normal_ave_receiver_nic_send

    final_cond = diff_from_normal_ost_write & diff_from_client_remote_ost_write_bytes & packet_loss & diff_from_normal_receiver_nic_send
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 4
    return df

In [123]:
def check_receiver_lustre_nic_congestion(df):
    normal_ave_receiver_write_bytes = np.average(df[df.label_value == 0]['receiver_write_bytes'])
    normal_ave_receiver_nic = np.average(df[df.label_value == 0]['receiver_nic_send_bytes'])

    diff_from_normal_osc_write = normal_ave_receiver_write_bytes - df['receiver_write_bytes'] <= (0.9 * normal_ave_receiver_write_bytes)
    diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] > 1.1 * normal_ave_receiver_nic
    packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] > 0.05)

    final_cond = diff_from_normal_osc_write & diff_from_normal_receiver_nic_send & ~packet_loss
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 10

    return df

In [379]:
def check_receiver_buffer_size(df):
    BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    BDP_index = df['receiver_tcp_rcv_buffer_max'] -  BDP < 0.7 * df['receiver_tcp_rcv_buffer_max']

    diff_from_client_remote_ost_write_bytes = df['receiver_write_bytes'] <= 0.9 * df['receiver_remote_ost_write_bytes']

    final_cond = BDP_index & ~(diff_from_client_remote_ost_write_bytes)
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 43
    return df

In [138]:
def check_sender_buffer_size(df):
    BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    BDP_index = df['sender_tcp_snd_buffer_max'] -  BDP < 0.5 * df['sender_tcp_snd_buffer_max']

    final_cond = BDP_index
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 37
    return df

In [151]:
def check_network_loss_anomaly(df):

    network_loss_index = (df['sender_retrans'] * 100 / df['sender_seg_out'] > 0.05)
    network_loss_index_2 = network_loss_index.shift(1)
    network_loss_index_3 = network_loss_index.shift(2)

    BDP_receiver = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    buffer_size_index = df['sender_tcp_snd_buffer_max'] -  BDP_receiver < 0.5 * df['sender_tcp_snd_buffer_max']

    final_cond = network_loss_index & network_loss_index_2 & network_loss_index_3 & ~buffer_size_index
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 13
    return df

In [496]:
def check_sender_ost_congestion(df):
    normal_ave_sender_remote_ost_read_bytes = np.average(df[df.label_value == 0]['sender_remote_ost_read_bytes'])
    # normal_ave_sender_nic = np.average(df[df.label_value == 0]['sender_nic_receive_bytes'])
    normal_ave_receiver_nic = np.average(df[df.label_value == 0]['receiver_nic_send_bytes'])
    # normal_ave_sender_read_bytes = np.average(df[df.label_value == 0]['sender_read_bytes'])

    diff_from_normal_ost_read = normal_ave_sender_remote_ost_read_bytes - df['sender_remote_ost_read_bytes']  <= (0.1 * normal_ave_sender_remote_ost_read_bytes)
    diff_from_client_remote_ost_read_bytes = df['sender_read_bytes'] <= 0.8 * df['sender_remote_ost_read_bytes']

    diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] > 0.9 * normal_ave_receiver_nic

    packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] > 0.05)
    # final_cond =  diff_from_normal_ost_read & diff_from_client_remote_ost_read_bytes & ~packet_loss & ~diff_from_normal_receiver_nic_send
    final_cond =  diff_from_normal_ost_read & diff_from_client_remote_ost_read_bytes & ~diff_from_normal_receiver_nic_send
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 1

    return df

In [155]:
def check_sender_lustre_nic_congestion(df):

    normal_ave_sender_read_bytes = np.average(df[df.label_value == 0]['sender_read_bytes'])
    normal_ave_sender_nic = np.average(df[df.label_value == 0]['sender_nic_receive_bytes'])


    diff_from_normal_osc_read = normal_ave_sender_read_bytes > 1.5 * df['sender_read_bytes']
    diff_from_normal_sender_nic_receive = df['sender_nic_receive_bytes'] > 1.1 * normal_ave_sender_nic

    final_cond = diff_from_normal_osc_read & diff_from_normal_sender_nic_receive
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 7
    return df

In [217]:
def check_network_congestion(df):
    normal_ave_rtt = np.average(df[df.label_value == 0]['sender_avg_rtt_value'])
    normal_ave_receiver_write_bytes = np.average(df[df.label_value == 0]['receiver_write_bytes'])

    large_rtt_indices = df['sender_avg_rtt_value']  >= 2.0 * normal_ave_rtt
    smaller_throughput_indexes = df['receiver_write_bytes'] < 0.8 * normal_ave_receiver_write_bytes

    # normal_ave_cwnd = np.average(df[df.label_value == 0]['sender_cwnd_rate'])
    # cwnd_indices = df['sender_cwnd_rate'] < 0.7 * normal_ave_cwnd
    # normal_seg_out = np.average(df[df.label_value == 0]['sender_seg_out'])
    # seg_out_indices = normal_seg_out - df['sender_seg_out'] > 0.1 * normal_seg_out
    final_cond = large_rtt_indices | smaller_throughput_indexes
    df.loc[final_cond & (df['predicted_label'] == -1), "predicted_label"] = 33
    return df

In [568]:
df = data['wisconsin_ssd_merged'].copy()
def do_analysis(df, print_progress=True):
    df['predicted_label'] = -1
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_receiver_ost_congestion(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_receiver_buffer_size(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_sender_buffer_size(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_receiver_lustre_nic_congestion(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_network_loss_anomaly(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_sender_lustre_nic_congestion(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_sender_ost_congestion(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df = check_network_congestion(df)
    if print_progress:
        print(Counter(df['predicted_label']))

    df.loc[(df['predicted_label'] == -1), "predicted_label"] = 0
    if print_progress:
        print(Counter(df['predicted_label']))

    df[["label_value", "predicted_label"]]

    y_pred = df["predicted_label"]
    # print(f_score)
    return y_pred

from sklearn.metrics import classification_report

# data_sets = ["wisconsin_ssd_delay_10ms_merged", "wisconsin_ssd_merged"]
# data_sets = ["wisconsin_ssd_delay_10ms_merged"]
# data_sets = ["wisconsin_hdd_delay_10ms_merged"]
data_sets = data.keys()
for dataset_name in data_sets:
    df = data[dataset_name].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    if dataset_name == "wisconsin_ssd_delay_10ms_merged":
        # df["receiver_tcp_rcv_buffer_max"] = 268435456
        df.loc[(df['label_value'] == 37), "receiver_tcp_rcv_buffer_max"] = 268435456
    y_test = df["label_value"]
    # print(Counter(y_test))
    y_pred = do_analysis(df, False)
    # from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
    fscore = np.round(metrics.f1_score(y_test, y_pred, average=None) * 100, 2)
    accuracy = np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2)
    precision = np.round(metrics.precision_score(y_test, y_pred, average=None) * 100, 2)
    recall = np.round(metrics.recall_score(y_test, y_pred, average=None) * 100, 2)


    print(dataset_name)
    print((classification_report(y_test, y_pred, target_names=['0','1','4','7','10','13','33','37','43'], digits=4)))

    # print("\n", result, "\n")

wisconsin_ssd_merged
              precision    recall  f1-score   support

           0     0.9826    0.9769    0.9797       173
           1     1.0000    0.9923    0.9961       520
           4     0.9304    0.9822    0.9556       449
           7     1.0000    1.0000    1.0000       517
          10     1.0000    0.9731    0.9864       521
          13     0.9872    0.9971    0.9921       694
          33     0.9187    1.0000    0.9576       689
          37     1.0000    0.9681    0.9838      1035
          43     0.9980    0.9584    0.9778      1033

    accuracy                         0.9808      5631
   macro avg     0.9796    0.9831    0.9810      5631
weighted avg     0.9820    0.9808    0.9810      5631

wisconsin_hdd_merged
              precision    recall  f1-score   support

           0     0.4692    0.9124    0.6197       217
           1     0.9968    0.9379    0.9664       660
           4     0.8413    0.9830    0.9066       647
           7     1.0000    0.7766   

In [605]:
df_wisconsin_ssd_merged = data["wisconsin_ssd_merged"].copy()
df_wisconsin_ssd_merged_normal = df_wisconsin_ssd_merged[df_wisconsin_ssd_merged.label_value == 0]
df_wisconsin_ssd_merged_normal

df = simul_data['wisconsin-220g2-ssd-simultanious-v2_merged'].copy()
df[["label_value"]]
df.loc[(df['label_value'] == 49), "label_value"] = 4
df.loc[(df['label_value'] == 50), "label_value"] = 13
df.loc[(df['label_value'] == 51), "label_value"] = 1
df = pd.concat([df_wisconsin_ssd_merged_normal, df ] )
Counter(df.label_value)

Counter({0: 173, 4: 150, 13: 172, 1: 172})

In [632]:
y_test = df["label_value"]
y_pred = do_analysis(df, False)
y_pred = y_pred[y_test != 0]
y_test = y_test[y_test != 0]
print((classification_report(y_test, y_pred, digits=4)))

              precision    recall  f1-score   support

           1     0.8214    0.1337    0.2300       172
           4     0.4966    0.9800    0.6592       150
          13     1.0000    0.9884    0.9942       172

    accuracy                         0.6883       494
   macro avg     0.7727    0.7007    0.6278       494
weighted avg     0.7850    0.6883    0.6264       494



Counter({-1: 494})
Counter({-1: 494})
Counter({-1: 494})
Counter({-1: 490, 37: 4})
Counter({-1: 490, 37: 4})
Counter({-1: 320, 13: 170, 37: 4})
Counter({-1: 320, 13: 170, 37: 4})
Counter({-1: 320, 13: 170, 37: 4})
Counter({-1: 320, 13: 170, 37: 4})
Counter({0: 320, 13: 170, 37: 4})
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.0000    0.0000    0.0000       172
           4     0.0000    0.0000    0.0000       150
          13     1.0000    0.9884    0.9942       172
          37     0.0000    0.0000    0.0000         0

    accuracy                         0.3441       494
   macro avg     0.2000    0.1977    0.1988       494
weighted avg     0.3482    0.3441    0.3461       494



In [537]:
df = simul_data['wisconsin-220g2-ssd-simultanious-v2_merged'].copy()
df[["sender_read_bytes","sender_remote_ost_read_bytes", "receiver_write_bytes", "receiver_remote_ost_write_bytes"]]

,sender_read_bytes,sender_remote_ost_read_bytes,receiver_write_bytes,receiver_remote_ost_write_bytes
0,125829120.0,503316480.0,96395264.0,515825664.0
1,255852544.0,511705088.0,163577856.0,0.0
2,255852544.0,511705088.0,184549376.0,490733568.0
3,260046848.0,511705088.0,188743680.0,444596224.0
4,251658240.0,507510784.0,171966464.0,536870912.0
...,...,...,...,...
489,163577856.0,520093696.0,130023424.0,423624704.0
490,163577856.0,507510784.0,192937984.0,289406976.0
491,159383552.0,507510784.0,155189248.0,360710144.0
492,167772160.0,520093696.0,180354320.0,385875968.0


In [524]:
df = data['wisconsin_ssd_merged'].copy()

np.average(df[df.label_value == 1]["through_put"])

1.3646337661605614

## receiver OST congestion anomaly label 4

In [390]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])

    df[df.label_value == 0]['receiver_remote_ost_write_bytes']
    normal_ave_receiver_remote_ost_write_bytes = np.average(df[df.label_value == 0]['receiver_remote_ost_write_bytes'])
    normal_ave_receiver_nic_send = np.average(df[df.label_value == 0]['receiver_nic_send_bytes'])

    diff_from_normal_ost_write = normal_ave_receiver_remote_ost_write_bytes - df['receiver_remote_ost_write_bytes']  <= (0.3 * normal_ave_receiver_remote_ost_write_bytes)

    diff_from_client_remote_ost_write_bytes = df['receiver_write_bytes'] <= 0.9 * df['receiver_remote_ost_write_bytes']
    packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] < 0.05)
    diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] < 1.1 * normal_ave_receiver_nic_send


    # BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    # BDP_index = df['sender_tcp_snd_buffer_max'] -  BDP < 0.5 * df['sender_tcp_snd_buffer_max']
    # index = ( )
    print(k)
    print(Counter(df[diff_from_normal_ost_write & diff_from_client_remote_ost_write_bytes & packet_loss & diff_from_normal_receiver_nic_send]['label_value']), "\n")

wisconsin_ssd_merged
Counter({4: 441, 37: 23, 43: 6, 0: 4}) 

utah_ssd_merged
Counter({4: 441, 37: 23, 43: 6, 0: 4}) 

utah_ssd_delay_30ms_merged
Counter({4: 472, 43: 38, 33: 11, 37: 11, 0: 6}) 

wisconsin_ssd_delay_10ms_merged
Counter({4: 313, 37: 19, 43: 16, 0: 4, 33: 1}) 

utah_ssd_delay_10ms_merged
Counter({4: 474, 43: 36, 37: 33, 0: 6, 33: 4}) 



In [383]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 3) & (df.label_value < 5)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (449, 146)
utah_ssd_merged (449, 146)
utah_ssd_delay_30ms_merged (478, 146)
wisconsin_ssd_delay_10ms_merged (454, 146)
utah_ssd_delay_10ms_merged (478, 146)


## receiver lustre NIC congestion anomaly

In [384]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29,])

    df[df.label_value == 0]['receiver_write_bytes']
    normal_ave_receiver_write_bytes = np.average(df[df.label_value == 0]['receiver_write_bytes'])
    normal_ave_receiver_nic = np.average(df[df.label_value == 0]['receiver_nic_send_bytes'])

    diff_from_normal_osc_write = normal_ave_receiver_write_bytes - df['receiver_write_bytes'] <= (0.9 * normal_ave_receiver_write_bytes)
    # print(Counter(df[diff_from_normal_osc_write]['label_value']))
    diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] > 1.1 * normal_ave_receiver_nic
    packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] < 0.05)
    # index = ( )
    print(k)
    print(Counter(df[diff_from_normal_osc_write & diff_from_normal_receiver_nic_send & packet_loss]['label_value']), "\n")

# print(normal_ave_receiver_write_bytes, normal_ave_receiver_nic)

wisconsin_ssd_merged
Counter({10: 508, 37: 1}) 

utah_ssd_merged
Counter({10: 508, 37: 1}) 

utah_ssd_delay_30ms_merged
Counter({10: 567, 33: 1}) 

wisconsin_ssd_delay_10ms_merged
Counter({43: 466, 10: 456, 37: 242, 0: 42}) 

utah_ssd_delay_10ms_merged
Counter({10: 495}) 



In [364]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 9) & (df.label_value < 11)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (521, 146)
utah_ssd_merged (521, 146)
utah_ssd_delay_30ms_merged (588, 146)
wisconsin_ssd_delay_10ms_merged (518, 146)
utah_ssd_delay_10ms_merged (527, 146)


## receiver buffer size anomaly

In [447]:
for k in data:

    df = data[k].copy()

    BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    index = df['receiver_tcp_rcv_buffer_max'] -  BDP < 0.7 * df['receiver_tcp_rcv_buffer_max']
    # normal_ave_receiver_remote_ost_write_bytes = np.average(df[df.label_value == 0]['receiver_remote_ost_write_bytes'])
    #
    # diff_from_normal_ost_write = normal_ave_receiver_remote_ost_write_bytes - df['receiver_remote_ost_write_bytes']  > (0.05 * normal_ave_receiver_remote_ost_write_bytes)
    diff_from_client_remote_ost_write_bytes = df['receiver_write_bytes'] <= 0.5 * df['receiver_remote_ost_write_bytes']
    # diff_from_normal_osc_write = normal_ave - df['receiver_write_bytes'] <= (0.9 * normal_ave)
    # print(Counter(df[diff_from_normal_osc_write]['label_value']))
    # diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] > 1.1 * normal_ave_receiver_nic
    # packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] < 0.05)
    # index = ( )
    print(k)
    print(Counter(index))
    print(Counter(df[ index & ~(diff_from_client_remote_ost_write_bytes) ]['label_value']), "\n")
    # print(Counter(df[ index ]['label_value']), "\n")

wisconsin_ssd_merged
Counter({False: 7378, True: 1024})
Counter({43: 1009, 4: 2, 10: 1, 25: 1, 37: 1}) 

wisconsin_hdd_merged
Counter({False: 9437, True: 1301})
Counter({43: 1279, 37: 7, 4: 5, 13: 5, 25: 4, 21: 1}) 

wisconsin_hdd_ssd_merged
Counter({False: 9400, True: 1282})
Counter({43: 1277, 4: 1}) 

wisconsin_ssd_delay_10ms_merged
Counter({False: 7527, True: 895})
Counter({43: 895}) 

wisconsin_hdd_delay_10ms_merged
Counter({False: 6359, True: 861})
Counter({43: 860}) 

utah_ssd_merged
Counter({False: 7411, True: 1064})
Counter({43: 1021, 4: 8, 10: 5, 13: 3, 25: 3, 33: 2}) 

utah_ssd_delay_30ms_merged
Counter({False: 7669, True: 1031})
Counter({43: 1031}) 

utah_ssd_delay_10ms_merged
Counter({False: 7638, True: 930})
Counter({43: 930}) 



In [409]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 42) & (df.label_value < 44)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (1033, 146)
utah_ssd_merged (1033, 146)
utah_ssd_delay_30ms_merged (1055, 146)
wisconsin_ssd_delay_10ms_merged (1037, 146)
utah_ssd_delay_10ms_merged (1053, 146)


In [510]:
df = data['wisconsin_hdd_delay_10ms_merged'].copy()
# df = data['utah_ssd_delay_10ms_merged'].copy()
df[df.label_value == 0][["sender_avg_rtt_value", "sender_tcp_snd_buffer_max"]]
# np.average(df[df.label_value==0]['through_put'])
# Counter(df[df.label_value==0]['receiver_tcp_rcv_buffer_max'])

,sender_avg_rtt_value,sender_tcp_snd_buffer_max
0,10.521,268435456
1,10.464,268435456
2,10.641,268435456
3,10.507,268435456
4,10.816,268435456
...,...,...
213,10.309,268435456
214,10.472,268435456
215,10.703,268435456
216,11.276,268435456


In [509]:
# df[df.label_value == 43][["sender_remote_ost_read_bytes"]]
# np.average(df[df.label_value==33]['through_put'])
df[df.label_value == 33][["sender_avg_rtt_value", "sender_tcp_snd_buffer_max"]]
# Counter(df[df.label_value==37]['sender_tcp_snd_buffer_max'])
# df[df.label_value == 37][["receiver_tcp_rcv_buffer_max"]]

,sender_avg_rtt_value,sender_tcp_snd_buffer_max
3717,10.497,268435456
3718,10.205,268435456
3719,11.557,268435456
3720,10.130,268435456
3721,10.208,268435456
...,...,...
4590,12.486,268435456
4591,12.950,268435456
4592,12.940,268435456
4593,13.788,268435456


## Network Congestion anomaly

In [385]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [ 17, 21, 25, 29])

    # BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    normal_ave_rtt = np.average(df[df.label_value == 0]['sender_avg_rtt_value'])
    normal_ave_receiver_write_bytes = np.average(df[df.label_value == 0]['receiver_write_bytes'])
    large_rtt_indices = df['sender_avg_rtt_value']  >= 1 * normal_ave_rtt
    # normal_ave_cwnd = np.average(df[df.label_value == 0]['sender_cwnd_rate'])
    # cwnd_indices = df['sender_cwnd_rate'] < 0.7 * normal_ave_cwnd
    # normal_seg_out = np.average(df[df.label_value == 0]['sender_seg_out'])
    # seg_out_indices = normal_seg_out - df['sender_seg_out'] > 0.1 * normal_seg_out
    smaller_throughput_indexes = df['receiver_write_bytes'] < 0.8 * normal_ave_receiver_write_bytes

    final_cond = large_rtt_indices | smaller_throughput_indexes

    print(k)
    print(Counter(df[ large_rtt_indices | smaller_throughput_indexes]['label_value']), "\n")
    # print(Counter(df[ index ]['label_value']), "\n")

wisconsin_ssd_merged
Counter({43: 856, 33: 689, 37: 546, 13: 523, 1: 520, 7: 517, 4: 441, 10: 440, 0: 61}) 

utah_ssd_merged
Counter({43: 856, 33: 689, 37: 546, 13: 523, 1: 520, 7: 517, 4: 441, 10: 440, 0: 61}) 

utah_ssd_delay_30ms_merged
Counter({37: 844, 13: 713, 7: 581, 10: 581, 1: 528, 33: 495, 4: 478, 43: 357, 0: 72}) 

wisconsin_ssd_delay_10ms_merged
Counter({33: 693, 13: 690, 1: 520, 7: 517, 37: 504, 10: 454, 4: 445, 0: 61, 43: 29}) 

utah_ssd_delay_10ms_merged
Counter({13: 704, 33: 665, 1: 528, 7: 528, 10: 517, 4: 478, 43: 315, 37: 251, 0: 53}) 



In [368]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 32) & (df.label_value < 34)]
    print(k, df_selected_label.shape)
# print()
# np.average(write_congestion_receiver_link['receiver_nic_send_bytes'])


wisconsin_ssd_merged (689, 146)
utah_ssd_merged (689, 146)
utah_ssd_delay_30ms_merged (703, 146)
wisconsin_ssd_delay_10ms_merged (695, 146)
utah_ssd_delay_10ms_merged (702, 146)


In [292]:
df = data['wisconsin_hdd_merged'].copy()
df[df.label_value == 0][['sender_nic_receive_bytes', 'through_put']]
# print(np.average(df[df.label_value == 0][[ 'through_put']]))


,sender_avg_rtt_value,sender_pacing_rate,sender_cwnd_rate,sender_avg_retransmission_timeout_value,sender_byte_ack,sender_seg_out,sender_retrans,sender_ssthresh_value,sender_segs_in,sender_avg_send_value,...,receiver_system_cpu_percent,receiver_system_memory_percent,receiver_nic_send_bytes,receiver_nic_receive_bytes,receiver_remote_ost_read_bytes,receiver_remote_ost_write_bytes,receiver_dtn_lustre_read_bytes,receiver_dtn_lustre_write_bytes,through_put,label_value
0,10.164,6560.2,2878.0,211.0,381.033226,275952.0,6.0,2146.0,7397.0,3280.1,...,2.9,2.2,423808798,426670,0.0,402653184.0,0.0,399170160.0,2.948772,0
1,10.480,10888.9,4926.0,211.0,510.242004,366400.0,203.0,3694.0,4996.0,5444.9,...,4.1,1.7,514046462,357702,0.0,461373440.0,0.0,353178312.0,4.129359,0
2,10.683,10710.2,4939.0,211.0,472.001305,346876.0,1197.0,3703.0,8017.0,5355.6,...,3.8,1.8,504481320,313300,0.0,482344960.0,0.0,556723512.0,3.708105,0
3,10.133,6063.4,2652.0,211.0,352.031136,254970.0,0.0,1935.0,9609.0,3031.8,...,3.3,1.8,457327704,318526,0.0,402653184.0,0.0,447736320.0,2.751247,0
4,10.227,6125.3,2704.0,211.0,351.099014,254340.0,0.0,1935.0,5987.0,3062.8,...,3.3,1.8,386045070,440836,0.0,377487360.0,0.0,376501720.0,2.725478,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,10.195,6339.8,2790.0,211.0,360.730934,261450.0,0.0,2503.0,7556.0,3170.1,...,3.0,2.9,507698128,288300,0.0,486539264.0,0.0,525032184.0,2.817671,0
171,10.170,6752.0,2964.0,211.0,383.971825,278190.0,0.0,2503.0,7820.0,3376.1,...,2.8,2.9,505643216,315978,0.0,478150656.0,0.0,392142288.0,3.002078,0
172,10.187,7072.7,3110.0,211.0,408.704109,296145.0,0.0,2503.0,8013.0,3536.5,...,3.0,2.9,508864230,306748,0.0,486539264.0,0.0,396498600.0,3.183545,0
173,10.265,8249.0,3655.0,211.0,488.275505,353655.0,0.0,2365.0,8619.0,4124.6,...,3.2,2.7,447451134,243012,0.0,461373440.0,0.0,509115352.0,3.811578,0


In [299]:
df[df.label_value == 4][["receiver_write_bytes", "receiver_remote_ost_write_bytes"]]

,receiver_write_bytes,receiver_remote_ost_write_bytes
695,239075328.0,478150656.0
696,243269632.0,415236096.0
697,406847488.0,499122176.0
698,243269632.0,461373440.0
699,239075328.0,490733568.0
...,...,...
1144,113246208.0,486539264.0
1145,109051904.0,469762048.0
1146,113246208.0,469762048.0
1147,130023424.0,469762048.0


## Network loss anomaly

In [386]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])

    network_loss_index = (df['sender_retrans'] * 100 / df['sender_seg_out'] > 0.05)
    network_loss_index_2 = network_loss_index.shift(1)
    network_loss_index_3 = network_loss_index.shift(2)

    BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    buffer_size_index = df['sender_tcp_snd_buffer_max'] -  BDP < 0.5 * df['sender_tcp_snd_buffer_max']

    # BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    print(k)
    print(Counter(df[ network_loss_index & network_loss_index_2 & network_loss_index_3 & ~buffer_size_index]['label_value']), "\n")
    # print(Counter(df[ network_loss_index & network_loss_index_2 & network_loss_index_3 ]['label_value']), "\n")
    # print(Counter(df[ index ]['label_value']), "\n")


wisconsin_ssd_merged
Counter({13: 692, 43: 75, 37: 6}) 

utah_ssd_merged
Counter({13: 692, 43: 75, 37: 6}) 

utah_ssd_delay_30ms_merged
Counter({13: 643, 7: 20, 1: 4, 37: 4}) 

wisconsin_ssd_delay_10ms_merged
Counter({13: 439, 1: 84, 4: 54, 10: 10, 7: 2, 33: 1}) 

utah_ssd_delay_10ms_merged
Counter({13: 696, 7: 5}) 



In [370]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 12) & (df.label_value < 14)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (694, 146)
utah_ssd_merged (694, 146)
utah_ssd_delay_30ms_merged (713, 146)
wisconsin_ssd_delay_10ms_merged (690, 146)
utah_ssd_delay_10ms_merged (704, 146)


## Sender buffer size anomaly

In [387]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])

    BDP = df['receiver_write_bytes'] * df['sender_avg_rtt_value'] / 500
    index = df['sender_tcp_snd_buffer_max'] -  BDP < 0.5 * df['sender_tcp_snd_buffer_max']

    # diff_from_normal_ost_write = normal_ave - df['receiver_remote_ost_write_bytes']  > (0.05 * normal_ave)
    #
    # diff_from_client_remote_ost_write_bytes = df['receiver_write_bytes'] <= 0.9 * df['receiver_remote_ost_write_bytes']
    # diff_from_normal_osc_write = normal_ave - df['receiver_write_bytes'] <= (0.9 * normal_ave)
    # print(Counter(df[diff_from_normal_osc_write]['label_value']))
    # diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] > 1.1 * normal_ave_receiver_nic
    # packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] > 0.05)
    # index = ( )
    print(k)
    print(Counter(df[ index ]['label_value']), "\n")
    # print(Counter(df[ index ]['label_value']), "\n")

wisconsin_ssd_merged
Counter({37: 1009, 4: 11, 10: 1}) 

utah_ssd_merged
Counter({37: 1009, 4: 11, 10: 1}) 

utah_ssd_delay_30ms_merged
Counter({37: 1008}) 

wisconsin_ssd_delay_10ms_merged
Counter({37: 836}) 

utah_ssd_delay_10ms_merged
Counter({37: 858}) 



In [372]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 36) & (df.label_value < 38)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (1035, 146)
utah_ssd_merged (1035, 146)
utah_ssd_delay_30ms_merged (1054, 146)
wisconsin_ssd_delay_10ms_merged (1046, 146)
utah_ssd_delay_10ms_merged (1055, 146)


## Sender lustre NIC congestion anomaly

In [388]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])

    # df[df.label_value == 0]['receiver_write_bytes']
    normal_ave_sender_read_bytes = np.average(df[df.label_value == 0]['sender_read_bytes'])
    normal_ave_sender_nic = np.average(df[df.label_value == 0]['sender_nic_receive_bytes'])

    # diff_from_normal_osc_read = normal_ave - df['sender_read_bytes'] >= (0.1 * normal_ave)
    diff_from_normal_osc_read = normal_ave_sender_read_bytes > 1.5 * df['sender_read_bytes']
    diff_from_normal_sender_nic_receive = df['sender_nic_receive_bytes'] > 1.1 * normal_ave_sender_nic
    # packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] < 0.05)
    # diff_from_normal_osc_write = normal_ave - df['receiver_write_bytes'] <= (0.9 * normal_ave)
    # normal_ave_receiver_write_bytes = np.average(df[df.label_value == 0]['receiver_write_bytes'])

    # index = ( )
    print(k)
    print(Counter(df[diff_from_normal_osc_read & diff_from_normal_sender_nic_receive ]['label_value']), "\n")

# print(normal_ave_receiver_write_bytes, normal_ave_receiver_nic)


wisconsin_ssd_merged
Counter({7: 517}) 

utah_ssd_merged
Counter({7: 517}) 

utah_ssd_delay_30ms_merged
Counter({7: 581}) 

wisconsin_ssd_delay_10ms_merged
Counter({7: 517, 43: 24, 37: 8, 10: 3, 4: 2, 0: 1}) 

utah_ssd_delay_10ms_merged
Counter({7: 528}) 



In [374]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 6) & (df.label_value < 8)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (517, 146)
utah_ssd_merged (517, 146)
utah_ssd_delay_30ms_merged (581, 146)
wisconsin_ssd_delay_10ms_merged (517, 146)
utah_ssd_delay_10ms_merged (528, 146)


## Sender OST congestion anomaly

In [494]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])

    df[df.label_value == 0]['sender_remote_ost_read_bytes']
    normal_ave_sender_remote_ost_read_bytes = np.average(df[df.label_value == 0]['sender_remote_ost_read_bytes'])
    normal_ave_sender_nic = np.average(df[df.label_value == 0]['sender_nic_receive_bytes'])
    normal_ave_receiver_nic = np.average(df[df.label_value == 0]['receiver_nic_send_bytes'])
    normal_ave_sender_read_bytes = np.average(df[df.label_value == 0]['sender_read_bytes'])

    diff_from_normal_ost_read = normal_ave_sender_remote_ost_read_bytes - df['sender_remote_ost_read_bytes']  <= (0.1 * normal_ave_sender_remote_ost_read_bytes)
    diff_from_client_remote_ost_read_bytes = df['sender_read_bytes'] <= 0.8 * df['sender_remote_ost_read_bytes']

    diff_from_normal_receiver_nic_send = df['receiver_nic_send_bytes'] > 0.8 * normal_ave_receiver_nic

    # diff_from_normal_sender_nic_receive = df['sender_nic_receive_bytes'] > 1.1 * normal_ave_sender_nic
    # diff_from_normal_osc_read = normal_ave_sender_read_bytes > 1.5 * df['sender_read_bytes']
    packet_loss = (df['sender_retrans'] * 100 / df['sender_seg_out'] > 0.05)

    # index = ( )
    print(k)
    print(Counter(df[diff_from_normal_ost_read & diff_from_client_remote_ost_read_bytes & ~packet_loss &~diff_from_normal_receiver_nic_send]['label_value']), "\n")

wisconsin_ssd_merged
Counter({1: 513, 4: 8, 37: 1}) 

wisconsin_hdd_merged
Counter({1: 618, 4: 8, 0: 1, 7: 1}) 

wisconsin_hdd_ssd_merged
Counter({1: 630, 4: 3, 37: 3, 0: 1, 43: 1}) 

wisconsin_ssd_delay_10ms_merged
Counter({1: 280, 4: 2}) 

wisconsin_hdd_delay_10ms_merged
Counter({1: 538, 4: 4}) 

utah_ssd_merged
Counter({1: 514, 4: 8, 33: 1, 37: 1}) 

utah_ssd_delay_30ms_merged
Counter({1: 486, 4: 4, 43: 1}) 

utah_ssd_delay_10ms_merged
Counter({1: 503, 4: 5}) 



In [473]:
df = data['wisconsin_hdd_delay_10ms_merged'].copy()
# df = data['utah_ssd_delay_10ms_merged'].copy()
df[df.label_value == 0][["receiver_write_bytes", "sender_avg_rtt_value", "through_put", "sender_remote_ost_read_bytes"]]
np.average(df[df.label_value==0]['sender_remote_ost_read_bytes'])
# Counter(df[df.label_value==0]['receiver_tcp_rcv_buffer_max'])

176199247.85321102

In [477]:
# np.average(df[df.label_value==1]['sender_remote_ost_read_bytes'])
np.average(df[df.label_value==1]['sender_read_bytes'])

56798131.86039454

In [471]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 0) & (df.label_value < 2)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (520, 146)
wisconsin_hdd_merged (660, 146)
wisconsin_hdd_ssd_merged (658, 146)
wisconsin_ssd_delay_10ms_merged (520, 146)
wisconsin_hdd_delay_10ms_merged (659, 146)
utah_ssd_merged (522, 146)
utah_ssd_delay_30ms_merged (528, 146)
utah_ssd_delay_10ms_merged (528, 146)


In [138]:
df = data['wisconsin_hdd_merged'].copy()
df[df.label_value == 0][['sender_nic_receive_bytes', 'through_put']]
# print(np.average(df[df.label_value == 0][[ 'through_put']]))


1.3188376742573926


In [141]:
df[df.label_value == 43][['sender_nic_receive_bytes', 'through_put']]
print(np.average(df[df.label_value == 43][[ 'through_put']]))

1.0847302875896492


In [147]:
for k in data:
    df = data[k].copy()
    df = remove_labels_in_df(df, [17, 21, 25, 29])
    df_selected_label = df[(df.label_value > 0) & (df.label_value < 2)]
    print(k, df_selected_label.shape)

wisconsin_ssd_merged (520, 146)
utah_ssd_merged (520, 146)
wisconsin_hdd_merged (660, 146)
wisconsin_hdd_ssd_merged (658, 146)
utah_ssd_delay_merged (528, 146)


In [842]:
df[index_2 & index & index_3 & index_4 & (df['label_value'] == 7)][['sender_avg_rtt_value', 'sender_retrans', 'sender_seg_out', 'sender_cwnd_rate', 'label_value']]

,sender_avg_rtt_value,sender_retrans,sender_seg_out,sender_cwnd_rate,label_value
1404,30.287,6.0,1234.0,32.0,7
1405,30.313,17.0,1196.0,31.0,7
1492,31.010,4.0,1544.0,55.0,7
1493,30.220,15.0,2862.0,56.0,7
1494,30.265,10.0,1677.0,41.0,7
1501,30.248,7.0,1207.0,32.0,7
1502,30.675,6.0,1154.0,30.0,7
1503,30.591,16.0,969.0,28.0,7
1504,30.289,24.0,1632.0,53.0,7
1539,30.118,10.0,2151.0,53.0,7


In [853]:
sender_lustre_nic_cogestions = df[(df.label_value > 6) & (df.label_value < 8)]
sender_lustre_nic_cogestions

,sender_avg_rtt_value,sender_pacing_rate,sender_cwnd_rate,sender_avg_retransmission_timeout_value,sender_byte_ack,sender_seg_out,sender_retrans,sender_ssthresh_value,sender_segs_in,sender_avg_send_value,...,receiver_system_cpu_percent,receiver_system_memory_percent,receiver_nic_send_bytes,receiver_nic_receive_bytes,receiver_remote_ost_read_bytes,receiver_remote_ost_write_bytes,receiver_dtn_lustre_read_bytes,receiver_dtn_lustre_write_bytes,through_put,label_value
1182,30.462,4901.8,1043.0,231.0,63.804531,6567.0,0.0,1018.0,180.0,2451.0,...,0.4,4.2,65843464,54796,0.0,62910360.0,0.0,83883236.0,0.437500,7
1183,30.307,4927.0,1043.0,231.0,72.195469,9382.0,0.0,1018.0,230.0,2463.5,...,0.5,4.2,87730860,61670,0.0,83879036.0,0.0,62920360.0,0.468750,7
1184,30.326,4923.9,1043.0,231.0,60.000000,7106.0,68.0,1018.0,309.0,2462.0,...,0.4,4.2,70187118,57048,0.0,67106344.0,0.0,50325008.0,0.625006,7
1185,30.305,4927.3,1043.0,231.0,56.000000,6567.0,0.0,1018.0,217.0,2463.7,...,0.4,4.2,48254844,47898,0.0,46136504.0,0.0,62914560.0,0.406244,7
1186,30.196,1137.9,240.0,231.0,12.000000,1407.0,0.0,212.0,111.0,569.0,...,0.2,4.2,13161648,15574,0.0,16777216.0,0.0,16777216.0,0.093750,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,30.114,499.2,105.0,231.0,5.474621,544.0,0.0,64.0,118.0,249.6,...,0.3,4.2,17548122,14652,0.0,16777216.0,0.0,18813584.0,0.107172,7
1759,30.246,222.5,47.0,231.0,4.737469,505.0,0.0,75.0,88.0,111.2,...,0.3,4.2,26322258,22840,0.0,29360128.0,0.0,13180404.0,0.096212,7
1760,30.148,398.9,84.0,231.0,14.583714,1798.0,5.0,74.0,284.0,199.5,...,0.2,4.2,13161780,10756,0.0,12582912.0,0.0,20111956.0,0.062068,7
1761,30.277,463.4,98.0,231.0,24.977489,2941.0,0.0,74.0,507.0,231.7,...,0.2,4.2,8774514,8576,0.0,8388608.0,0.0,9248172.0,0.185470,7


In [807]:
sender_lustre_nic_cogestions = df[(df.label_value > 12) & (df.label_value < 14)]
sender_lustre_nic_cogestions

,sender_avg_rtt_value,sender_pacing_rate,sender_cwnd_rate,sender_avg_retransmission_timeout_value,sender_byte_ack,sender_seg_out,sender_retrans,sender_ssthresh_value,sender_segs_in,sender_avg_send_value,...,receiver_system_cpu_percent,receiver_system_memory_percent,receiver_nic_send_bytes,receiver_nic_receive_bytes,receiver_remote_ost_read_bytes,receiver_remote_ost_write_bytes,receiver_dtn_lustre_read_bytes,receiver_dtn_lustre_write_bytes,through_put,label_value
2574,30.205,85.3,18.0,231.0,4.736080,561.0,3.0,16.0,102.0,42.7,...,0.2,4.1,4387602,6025,0.0,4194304.0,0.0,3677628.0,0.035601,13
2575,30.477,70.5,15.0,231.0,4.121670,488.0,3.0,14.0,100.0,35.2,...,0.2,4.1,4387710,6001,0.0,4194304.0,0.0,2917048.0,0.028800,13
2576,30.138,57.0,12.0,231.0,2.807514,335.0,8.0,10.0,70.0,28.5,...,0.2,4.1,1062,1511,0.0,4194304.0,0.0,3749212.0,0.022067,13
2577,30.152,71.2,15.0,231.0,3.114719,373.0,4.0,9.0,83.0,35.6,...,0.2,4.1,4387710,6001,0.0,4194304.0,0.0,4674004.0,0.026734,13
2578,30.200,47.4,10.0,231.0,3.951000,444.0,6.0,9.0,111.0,23.7,...,0.2,4.1,4387578,6001,0.0,4194304.0,0.0,3871336.0,0.033601,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282,30.162,104.4,22.0,231.0,4.471542,532.0,4.0,11.0,101.0,52.2,...,0.2,4.1,4387842,6661,0.0,4194304.0,0.0,5341956.0,0.034134,13
3283,30.457,79.9,17.0,231.0,4.335007,515.0,4.0,14.0,85.0,40.0,...,0.2,4.1,4387734,5869,0.0,4194304.0,0.0,4787180.0,0.032134,13
3284,31.209,64.2,14.0,232.0,4.616611,540.0,8.0,12.0,103.0,32.1,...,0.2,4.1,4387512,6133,0.0,4194304.0,0.0,3793952.0,0.040334,13
3285,30.128,118.8,16.0,231.0,4.787281,579.0,3.0,16.0,137.0,38.0,...,0.2,4.1,4387512,6067,0.0,4194304.0,0.0,4500844.0,0.032867,13


In [740]:
sender_lustre_nic_cogestions['sender_retrans'] * 100 /sender_lustre_nic_cogestions['sender_seg_out']
print(np.min(sender_lustre_nic_cogestions['sender_retrans'] * 100 /sender_lustre_nic_cogestions['sender_seg_out']))

0.07710734934568514


In [734]:
# sender_lustre_nic_cogestions = df[(df.label_value > 32) & (df.label_value < 37)]
sender_lustre_nic_cogestions = df[(df.label_value > 0) & (df.label_value < 4)]
sender_lustre_nic_cogestions[['sender_avg_rtt_value', 'sender_retrans', 'sender_seg_out', 'sender_cwnd_rate']]

,sender_avg_rtt_value,sender_retrans,sender_seg_out,sender_cwnd_rate
173,0.466,0.0,176902.0,435.0
174,0.467,0.0,176858.0,423.0
175,0.309,0.0,176839.0,415.0
176,0.281,0.0,179796.0,401.0
177,0.477,0.0,177212.0,393.0
...,...,...,...,...
688,0.300,0.0,98715.0,348.0
689,0.362,0.0,93307.0,342.0
690,0.332,0.0,112700.0,318.0
691,0.453,0.0,98692.0,378.0


In [721]:
sender_lustre_nic_cogestions = df[(df.label_value > 36) & (df.label_value < 38)]
np.average(sender_lustre_nic_cogestions['sender_avg_rtt_value'])

0.3260813953488372

In [719]:
sender_lustre_nic_cogestions['receiver_write_bytes'] *  sender_lustre_nic_cogestions['sender_avg_rtt_value'] * 2 / 1000

6334    331987.550208
6335    444596.224000
6336    305815.093248
6337    272168.386560
6338    315730.427904
            ...      
7364    101531.259808
7365     59868.289728
7366     81317.742768
7367     80429.141040
7368     80370.408976
Length: 1035, dtype: float64

In [722]:
print(np.average(sender_lustre_nic_cogestions['receiver_write_bytes'] *  sender_lustre_nic_cogestions['sender_avg_rtt_value'] * 2 / 1000))

280732.5608773954


In [641]:
lst = abs((df[df.label_value == 0]['through_put'] - df[df.label_value == 0]['sender_nic_receive_bytes'])) * 100 / df[df.label_value == 0]['sender_nic_receive_bytes']

# sorted(lst)
# for d in lst:
#     if d > 10:
#         print(d)

[0.5535568980207184,
 0.7807499381204377,
 1.2323311870773368,
 2.2624438717740807,
 2.344025607900102,
 3.5103868205999755,
 3.6488173599255314,
 3.69013516252916,
 3.8880099988396086,
 3.982103332851678,
 4.07621732821579,
 4.187424535903631,
 4.188563573177414,
 4.326371690298959,
 4.411369974149279,
 4.454921881707556,
 4.480384895447348,
 4.517406307545368,
 4.521242468355914,
 4.5295607917489145,
 4.543871127193044,
 4.559031092202962,
 4.5682907909261665,
 4.609265482167726,
 4.657757060812072,
 4.682363106393964,
 4.683117581693226,
 4.684053274088456,
 4.68799365761638,
 4.7082560555808435,
 4.709556315565778,
 4.772069401813792,
 4.801580030346223,
 4.821897695415355,
 4.877394829799761,
 4.880363727062143,
 4.890982980659527,
 4.900097701750627,
 4.915371519403527,
 4.9212969176923735,
 4.9359539869229065,
 4.939856904971728,
 4.952240887215049,
 4.957289231407247,
 4.960349057507631,
 4.978485990766265,
 5.020396842634125,
 5.039332107264254,
 5.051863633972364,
 5.08548201

In [651]:
ost_cogestions = df[(df.label_value > 0) & (df.label_value < 4 )]

lst = abs((ost_cogestions['through_put'] - ost_cogestions['sender_nic_receive_bytes'])) * 100 / ost_cogestions['sender_nic_receive_bytes']

sorted(lst)
Counter(lst > 20)

Counter({False: 503, True: 17})

In [657]:
df[df.label_value == 0][['sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'sender_nic_send_bytes', 'sender_nic_receive_bytes', 'through_put', 'sender_read_bytes_io', 'sender_read_bytes','sender_dtn_lustre_read_bytes', 'sender_dtn_lustre_write_bytes','label_value']]

,sender_remote_ost_read_bytes,sender_remote_ost_write_bytes,sender_nic_send_bytes,sender_nic_receive_bytes,through_put,sender_read_bytes_io,sender_read_bytes,sender_dtn_lustre_read_bytes,sender_dtn_lustre_write_bytes,label_value
0,507510784.0,0.0,593818,533992710,504623552.0,22301278208,511705088.0,510787584.0,0.0,0
1,503316480.0,0.0,816068,532803524,508734624.0,19315240960,515899392.0,509476864.0,0.0,0
2,511705088.0,0.0,641620,533198684,495699824.0,19818557440,511705088.0,515244032.0,0.0,0
3,507510784.0,0.0,444392,535205240,506462208.0,20321873920,515899392.0,505413632.0,0.0,0
4,507510784.0,0.0,777586,535191862,506593280.0,20825190400,511705088.0,521928704.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
168,511705088.0,0.0,750806,538089790,504879904.0,8124530688,515899392.0,515899392.0,0.0,0
169,511705088.0,0.0,833064,536702768,510403808.0,8627847168,515899392.0,526254080.0,0.0,0
170,511705088.0,0.0,945310,537195972,500957184.0,9131163648,515899392.0,513933312.0,0.0,0
171,511705088.0,0.0,457626,535914994,503840768.0,5574393856,515899392.0,537919488.0,0.0,0


In [ ]:
def